# Step 1: Setting up Yolov8

First, we’ll need to install the Ultralytics package using the following command.


In [2]:
!pip install ultralytics

Then, we will import the necessary libraries and install the Yolov8 model to our local system.

In [32]:
# import ultralytics
from ultralytics import YOLO

# Configure the tracking parameters and run the tracker
model = YOLO('yolov8n.pt')


In [33]:
!nvidia-smi

Thu Mar 14 21:51:48 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.76                 Driver Version: 551.76         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   43C    P8              5W /   30W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Step 2: Defining Queue lines

Here we use the [polygonzone](https://roboflow.github.io/polygonzone/) website, that is developed by [roboflow](https://www.bing.com/search?q=roboflow&cvid=9bf5f509043f4ee89e6e3ccb83a5b8c6&gs_lcrp=EgZjaHJvbWUqBAgBEAAyBggAEEUYPDIECAEQADIECAIQADIECAMQADIECAQQADIECAUQADIGCAYQRRg8MgYIBxBFGDwyBggIEEUYPNIBCDQ1NjNqMGo0qAIAsAIA&FORM=ANAB01&PC=U531), to get the coordinates of the queue lines of our retail store. To draw the polygon make sure the mode is Polygon and then start from a point draw the required shape and end back to the same point.After this the coordinates will be reflected in the box beside from where you can copy and use it as your ROI.

![Image Description](other/Picture1.png)

Now, let’s go through a sample implementation to see how object tracking would work for retail queue monitoring.

# Step 3: Setting Up YOLOv8 and Video Processing

In this portion of code, we set up the components required for object tracking and data gathering. Loading the YOLOv8 model, opening a video clip, establishing areas of interest (ROI), and building data structures for tracking and time recording are all part of this process. We're also setting up a CSV file and writer, as well as frame counts for video processing. This code serves as the foundation for our data collection and object tracking duties.

In [37]:
import cv2
import numpy as np
from ultralytics import YOLO
import csv

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")

# Open the video file
video_path = "Videos/Shopping, People, Commerce, Mall, Many, Crowd, Walking   Free Stock video footage   YouTube.mp4"
cap = cv2.VideoCapture(video_path)

fps = 20

# Define the output video path
output_video_path = "/Videos/Output/output.mp4"

# Define the codec and create a VideoWriter object for the output video
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, 25.0, (frame_width, frame_height))

# Define ROI (Region of Interest) coordinates for the rectangular regions
roi1_coords = np.array([[747, 622],[707, 38],[807, 22],[931, 654],[747, 622]], dtype=np.int32)

roi2_coords = np.array([[1039, 62],[1243, 546],[1271, 502],[1231, 286],[1107, 34],[1039, 62]], dtype=np.int32)

people_enter_queue = {}
timespent = []

filename = "queue_time.csv"

# Open the file in write mode
file = open(filename, 'w', newline='')

# Create a CSV writer object
csv_writer = csv.writer(file)

frame_count =0

# Step 4:Video processing Loop

We're going into the video frames in this section of the code, keeping a careful eye on specific areas of interest. We want to detect and track items within these regions that are most likely individuals as they move through the frame. We're tracking when individuals enter and exit these places, as well as the time they spend waiting in queue. 

In [38]:
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        annotated_frame = frame.copy()

        # Draw the ROIs
        cv2.drawContours(annotated_frame, [roi1_coords], -1, (255, 0, 0), 3)
        cv2.drawContours(annotated_frame, [roi2_coords], -1, (255, 0, 0), 3)

        # Run YOLOv8 tracking on the original frame
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xyxy.cpu()

        print("Found: ", people_enter_queue)

        if results[0].boxes.id is not None:
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Check if the center of bounding boxes is inside the ROI
            for box, track_id in zip(boxes, track_ids):
                print("Tracking:", track_id)

                x1, y1, x2, y2 = box
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                x = (x1 + x2) / 2
                y = (y1 + y2) / 2

                # Visualize the people being tracked in queues on the frame
                if ((cv2.pointPolygonTest(roi1_coords, (x, y), False) > 0) or (
                        (cv2.pointPolygonTest(roi2_coords, (x, y), False)) > 0)):
                    if str(track_id) not in people_enter_queue:
                        # Get timestamp
                        people_enter_queue[str(track_id)] = str(frame_count)

                    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(annotated_frame, "Person id:" + str(track_id), (x1, y1 - 5),
                                cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (20, 255, 0), 2)
                else:
                    print("outside:", track_id)

                    if str(track_id) in people_enter_queue:
                        # Get timestamp
                        exit = frame_count

                        # Get first timestamp
                        start = people_enter_queue[str(track_id)]

                        time_spent = (exit - int(start)) / fps
                        print("time spent ", time_spent, "by person", track_id)

                        timespent.append(time_spent)

                        # Write string to the file
                        csv_writer.writerow(["Time spent by person " + str(track_id) + " in line is " + str(time_spent)])

                        people_enter_queue.pop(str(track_id))

        # Write the frame to the output video
        out.write(annotated_frame)
        frame_count += 1

    else:
        # Break the loop if there are no more frames
        break

0: 384x640 13 persons, 1 bird, 99.7ms
Speed: 4.9ms preprocess, 99.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Found:  {}
Tracking: 1
outside: 1
Tracking: 2
outside: 2
Tracking: 3
outside: 3
Tracking: 4
outside: 4
Tracking: 5
outside: 5
Tracking: 6
Tracking: 7
outside: 7
Tracking: 8
outside: 8
Tracking: 9
Tracking: 10
outside: 10
Tracking: 11
outside: 11
Tracking: 12
Tracking: 13
outside: 13
Tracking: 14
outside: 14
0: 384x640 13 persons, 1 bird, 103.2ms
Speed: 3.3ms preprocess, 103.2ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
Found:  {'6': '0', '9': '0', '12': '0'}
Tracking: 1
outside: 1
Tracking: 2
outside: 2
Tracking: 3
outside: 3
Tracking: 4
outside: 4
Tracking: 5
outside: 5
Tracking: 6
Tracking: 7
outside: 7
Tracking: 8
outside: 8
Tracking: 9
Tracking: 10
outside: 10
Tracking: 11
outside: 11
Tracking: 12
Tracking: 13
outside: 13
Tracking: 14
outside: 14
0: 384x640 15 persons, 1 bird, 95.2ms
Speed: 2.3ms preprocess, 95.2ms inference, 

# Step 5: Time calculation

We use the formula time_spent = (exit - start) / fps in this code section to compute the time individuals spent in a queue by iterating through the people_enter_queue dictionary. We then use the formula average = sum(time spent) / len(time spent) to calculate the average time spent by all persons. When the loop is over, the program carefully handles resource release, completing video processing.

In [39]:
for person in people_enter_queue:
    #get timestamp
    exit = frame_count

    #get first timestamp
    start = people_enter_queue.get(person)

    time_spent = (exit - int(start))/fps
    print("time spent ", time_spent, "by person", person)

    timespent.append(time_spent)

    # Write string to the file
    csv_writer.writerow(["Time spent by person "+ str(person)+" in line is "+str(time_spent)])

    average = sum(timespent)/len(timespent)

    print("Average of list: ", round(average,3))

    csv_writer.writerow(["Average time spent in line is "+str(round(average,3))])
    # Break the loop if the end of the video is reached
    break


# Release the video capture object, release the output video, and close the display window
cap.release()
out.release()
cv2.destroyAllWindows()
file.close()

print(f"Output video saved at: {output_video_path}")

time spent  17.05 by person 9
Average of list:  5.294
Output video saved at: /Videos/Output/output.mp4


# Output

Here is the final csv file displaying the input the time spent by the person in the queue and average time.

![Image Description](other/Picture2.png)

Here is the final tracking [video](https://drive.google.com/file/d/1pSHGDGw4_7CvKfczNPQBlKum0ixXI0b9/view?usp=drive_link)

![Image Description](other/Picture3.png)

# Conclusion

This article delves into the interesting world of queue analytics via object detection. It's a fantastic chance for merchants to capitalise on the power of these advancements as they become more accessible and affordable. Consider having unprecedented insights into customer behaviour, allowing you to make data-driven decisions that improve in-store experiences. In this fiercely competitive multichannel era, using computer vision and future technologies has become a strategic essential for shops seeking to not only survive but grow.